In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from env import *
from wrangle import *
import re

import wordninja

# Get Data

In [2]:
sql_db = "zillow"
query = "SELECT * FROM properties_2017 JOIN predictions_2017 USING(parcelid) WHERE (`propertylandusetypeid` = 261) & (YEAR(`transactiondate`) = 2017);"
df = get_data(sql_db,query)

Reading CSV


# Clean Data

### Nulls

In [ ]:
df.isnull().sum()

In [4]:
df = handle_missing_values(df,.7,.99)

In [5]:
df.isnull().sum()

parcelid                        0
id                              0
bathroomcnt                     0
bedroomcnt                      0
calculatedbathnbr               0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
id.1                            0
logerror                        0
transactiondate                 0
dtype: int64

In [6]:
df.columns

Index(['parcelid', 'id', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'id.1', 'logerror',
       'transactiondate'],
      dtype='object')

### Outliers

In [7]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [8]:
def get_lower_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the lower outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the lower bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    return s.apply(lambda x: max([lower_bound - x, 0]))

In [9]:
curr_df = pd.DataFrame()
k=1.5

In [10]:
for col in df.select_dtypes('number'):
        curr_df[col + '_lower_outliers'] = get_lower_outliers(df[col], k)

In [11]:
for col in df.select_dtypes('number'):
        curr_df[col + '_upper_outliers'] = get_upper_outliers(df[col], k)

### Rename Columns

In [12]:
for column in df.columns:
    print(("_".join(wordninja.split(column))))

parcel_id
id
bathroom_cnt
bedroom_cnt
calculated_bath_n_br
calculated_finished_square_feet
finished_square_feet_12
fips
full_bath_cnt
latitude
longitude
lot_size_square_feet
property_county_land_use_code
property_land_use_type_id
raw_census_tract_and_block
region_id_city
region_id_county
region_id_zip
room_cnt
year_built
structure_tax_value_dollar_cnt
tax_value_dollar_cnt
assessment_year
land_tax_value_dollar_cnt
tax_amount
census_tract_and_block
id_1
log_error
transaction_date


# Get Dummies

### Categorical Columns

In [13]:
num,cat=separate_column_type_list(df)

In [14]:
longest_column_name = max([len(i) for i in cat])
for i in cat:
    print(f"Discrete: {i:>{longest_column_name}} = {df[i].nunique()}")

Discrete: propertycountylandusecode = 22
Discrete:     propertylandusetypeid = 1
Discrete:            regionidcounty = 3
Discrete:           transactiondate = 257


# Split Data

In [18]:
train.columns

Index(['parcelid', 'id', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock', 'id.1', 'logerror',
       'transactiondate'],
      dtype='object')

In [15]:
train,val,test = split_data(df)

In [19]:
X_train,y_train = isolate_target(train,target='structuretaxvaluedollarcnt')

In [25]:
num,cat = separate_column_type_list(df)

In [37]:
cat

['propertycountylandusecode',
 'propertylandusetypeid',
 'regionidcounty',
 'transactiondate']

In [36]:
def unique_id_columns(df):
    unique_ids = []
    for col in df.columns:
        unique_count = df[col].nunique()
        if unique_count/len(df) > 0.99:
            unique_ids.append(col)
    return unique_ids

In [40]:
unique_columns_list = unique_id_columns(df[num])

In [41]:
longest_column_name = max([len(i) for i in unique_columns_list])
for i in unique_columns_list:
    print(f"Unique: {i:>{longest_column_name}} = {df[i].nunique()}")

Unique: parcelid = 50560
Unique:       id = 50560
Unique:     id.1 = 50673
Unique: logerror = 50428


In [35]:
df['parcelid'].nunique()/df.shape[0]

0.9977700155901565

In [32]:
1-(df['parcelid'].nunique()/df.shape[0])

0.002229984409843455

In [24]:
scale_data(X_train)

array([[1.72664626e-03, 3.25773917e-01, 0.00000000e+00, ...,
        5.18124899e-06, 7.98020925e-01, 4.16528026e-01],
       [7.91690428e-04, 6.90692068e-01, 0.00000000e+00, ...,
        6.26817229e-07, 6.82871205e-01, 4.04646269e-01],
       [2.47778598e-02, 9.94360201e-01, 1.25000000e-01, ...,
        5.18880190e-04, 4.71112715e-01, 4.16892059e-01],
       ...,
       [9.54228872e-03, 1.77143405e-01, 0.00000000e+00, ...,
        8.52677976e-06, 7.51713653e-01, 4.00989286e-01],
       [2.98069141e-03, 9.59221877e-02, 1.66666667e-01, ...,
        1.93825523e-05, 1.70849869e-02, 4.25452619e-01],
       [6.28128003e-03, 4.91598823e-01, 5.00000000e-01, ...,
        3.81847221e-06, 3.14423027e-01, 4.20570326e-01]])

### Visualize

In [ ]:
for col in cat:
    if (df[col].nunique()<=10) & (df[col].nunique()>1):
        sns.histplot(df[col])
        plt.show()